# AfricaAIED 2024 Hackathon
AfricaAIED 2024 Hackathon 🚀
Welcome to the AfricaAIED 2024 Hackathon! This notebook is designed to help you get started with answering scientific riddles from the National Science and Maths Quiz (NSMQ) using the Mistral-7B-Instruct model. The setup is similar to what we used to compete with students during the finals of the 2023 NSMQ, excluding the confidence modeling pipeline.

You are provided with a sample riddle to experiment with, but feel free to experiment with riddles of your own. 🤔🧠

Happy Building, and best of luck in the Hackathon. We can't wait to see what you build. 🎉🏆

P.S. This notebook will run fine on a T4 GPU, albeit slowly. For optimal performance, consider using an A100.

In [1]:
# Install dependencies
!pip install -q transformers
!pip install -q accelerate
!pip install -q einops
!pip install -q langchain
!pip install -q langchain-community
!pip install -q bitsandbytes
!pip install -q xformers

In [5]:
# To use the Mistral-7B-Instruct-v0.1 model from Mistral-AI,
# 1. Request for access to the model from: https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1
# 2. Create an Access Token from your HuggingFace account, if you don't have one already.
# 3. Add the Access Token to your colab secrets under the name "HF_TOKEN"
import os
from google.colab import userdata


os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [3]:
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [ ]:
# Fetch Mistral-7B-Instruct-v0.1 model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)


pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=512,
        temperature = 0.7,
        do_sample=False,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        torch_dtype=torch.float16,
)

llm = HuggingFacePipeline(pipeline=pipe)

In [30]:
PROMPT_TEMPLATE = template = """  <s>[INST]   You are a science prodigy currently competing in a National Science competition. \
        You are now in the fifth round, where you must provide a one-word answer to a riddle. \
        Remember, your answer should consist of just the term the riddle is pointing to, and nothing else. \
        Adding additional text will result in point deductions.
        Here's an example to guide you: \
        Riddle: You might think I am a rather unstable character because I never stay at one place. \
                However my motion obeys strict rules and I always return to where I started and even if I have to leave that spot again I do it in strict accordance to time. \
                I can be named in electrical and mechanical contexts. \
                In all cases i obey the same mathematical rules. \
                In order to fully analyse me you would think about a stiffness or force constant restoring force and angular frequency. \
        Answer: oscillator

        Read the riddle below and provide the correct answer.

        Riddle: {riddle}
        Answer:
        [/INST] </s>
     """
def answer_with_mistral(riddle):
    prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["riddle"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    answer = llm_chain.run({"riddle":riddle})

    return answer

In [31]:
sample_riddle = """
    I am a set of points.
    I am the set of all possible positions a point can take.
    I may be a region in a plane or in space, but more often I am a continuous curve
    I am the path followed by a point subject to some constraint. Who am I?
"""

# Generate an answer for sample riddle assuming all clues have been read.
# P.S. This will take some time to run if you are running on a T4 GPU.
generated_answer = answer_with_mistral(riddle=sample_riddle)
print(generated_answer)